In [1]:
import os
import pandas as pd
import natsort
data_dir =  './crop_images'
data_per = os.listdir(data_dir)

In [2]:
def max_frame(data_per):
    max_frames = 0
    for person in data_per:
        if person == '.DS_Store':
            continue
        words_dir = f'{data_dir}/{person}'
        Id = os.listdir(words_dir)
        for id in Id :
            id_dir = f'{data_dir}/{person}/{id}'
            num = os.listdir(id_dir)
            for n  in num:
                #print("n is", n)
#                 words_dir_2 = f'{data_dir}/{person}/{id}/{n}/crop_face'
                words_dir_2 = f'{data_dir}/{person}/{id}/{n}/crop_lip'
                frames = os.listdir(words_dir_2)
                #print(len(frames))
                max_frames = max(len(frames), max_frames)
    return max_frames

In [3]:
def concat_x_data(x_data, x_data_):
    if x_data_.shape[0] == max_frame(data_per):
        return np.concatenate([x_data, x_data_], axis = 1) 
    else: 
        print(max_frame(data_per))
        pad = max_frame(data_per) - x_data_.shape[1]
        print("pad for ",pad)
        zero_pad = np.zeros((1, pad, 64, 64, 3))
        print(zero_pad.shape)
#         print(x_data_.shape)
        data = np.concatenate([x_data_, zero_pad], axis = 1) #padding
        return data

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class DataAugmentation:
    @classmethod
    def generate(self, img):
        img_array = image.array_to_img(img)
        samples = np.expand_dims(img_array, axis=0)
        
        # generatior 생성
        datagen = ImageDataGenerator(
            width_shift_range=[-10,10], 
            height_shift_range=0.5, 
            rotation_range=45)
        
        it = datagen.flow(samples, batch_size=1)
        augmented = it.next()
        img_augmented = augmented[0]
        img_augmented /= 255. 
        img_augmented = np.expand_dims(img_augmented, 0)

        return img_augmented

In [5]:
from tensorflow.keras.preprocessing import image
import numpy as np
x_data = np.empty(5)
for person in natsort.natsorted(data_per):
    if person == '.DS_Store':
        continue
    #phrases_dir = f'{data_dir}/{person}/phrases'
    words_dir = f'{data_dir}/{person}'
    Id = os.listdir(words_dir)
    for sentence in natsort.natsorted(Id) :
        id_dir = f'{data_dir}/{person}/{sentence}'
        num = os.listdir(id_dir)
        for n  in natsort.natsorted(num):
            print("person: {}, sentence: {}, n: {} ".format(person, sentence, n))
#             words_dir_2 = f'{data_dir}/{person}/{sentence}/{n}/crop_face'
            words_dir_2 = f'{data_dir}/{person}/{sentence}/{n}/crop_lip'
            frames = os.listdir(words_dir_2)
            x_data_ = np.empty(4)
            x_data_aug = np.empty(4)
            isFirst = True
            for frame in natsort.natsorted(frames):
                img_path = f'{words_dir_2}/{frame}'
                
                # flatten 제거한 frame processing
                img = image.load_img(img_path, target_size=(64, 64))
                img_tensor = image.img_to_array(img) #tensor로 type 변경
                img_tensor /= 255. # scaling 
                img_tensor = np.expand_dims(img_tensor, axis = 0) # (1, 64, 64, 3) 차원
                
                # frame 별 묶기
                if isFirst:
                    x_data_ = img_tensor
                    x_data_aug = DataAugmentation().generate(img)
                    isFirst = False
                else:
                    x_data_ = np.concatenate([x_data_, img_tensor], axis = 0)
                    img_augmetned = DataAugmentation.generate(img)
                    x_data_aug = np.concatenate([x_data_aug, img_augmetned], axis=0)
                
                print(x_data_.shape)
                
            # 이미지 전체 통합시키기
            x_data_ = np.expand_dims(x_data_, axis = 0)  #[1, frame, 64, 64, 3]
            x_data_aug = np.expand_dims(x_data_aug, axis=0)
            if (n == '01' and sentence == '01' and person == 'F01'):
                x_data = concat_x_data(x_data, x_data_)
                y_data = np.array([int(sentence)])
                data_aug = concat_x_data(x_data, x_data_aug)
                x_data = np.concatenate([x_data, data_aug], axis = 0)
                y_data = np.concatenate([y_data, [int(sentence)]])
            else:
                if x_data_.shape[0] == max_frame(data_per):
                    x_data = concat_x_data(x_data, x_data_)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                    x_data = concat_x_data(x_data, x_data_aug)
                    y_data = np.concatenate([y_data, [int(sentence)]])y
                    
                else: 
                    data = concat_x_data(x_data, x_data_)
                    x_data = np.concatenate([x_data, data], axis = 0)
                    y_data = np.concatenate([y_data, [int(sentence)]])
                    data_aug = concat_x_data(x_data, x_data_aug)
                    x_data = np.concatenate([x_data, data_aug], axis = 0)
                    y_data = np.concatenate([y_data, [int(sentence)]])
            print(x_data.shape)
            print("y.shape: {}" .format(y_data.shape))

person: F01, sentence: 01, n: 01 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
(2, 27, 64, 64, 3)
y.shape: (2,)
person: F01, sentence: 01, n: 02 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
27
pad for  14
(1, 14, 64, 64, 3)
27
pad for  14
(1, 14, 64, 64, 3)
(4, 27, 64, 64, 3)
y.shape: (4,)
person: F01, sentence: 01, n: 03 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
27
pad for  14
(1, 14, 64, 64, 3)
27
pad for  14
(1, 14, 64, 64, 3)
(6, 27, 64, 64, 3)
y.shape: (6,)
person: F01, s

27
pad for  15
(1, 15, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
(56, 27, 64, 64, 3)
y.shape: (56,)
person: F01, sentence: 03, n: 09 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
(58, 27, 64, 64, 3)
y.shape: (58,)
person: F01, sentence: 03, n: 10 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
27
pad for  16
(1, 16, 64, 64, 3)
27
pad for  16
(1, 16, 64, 64, 3)
(60, 27, 64, 64, 3)
y.shape: (60,)
person: F01, sentence: 04, n: 01 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
27
pad for  21
(1, 21, 64, 64, 3)
27
pad for  21
(1, 21, 64, 64, 3)
(62, 27, 64, 64, 3)
y.shape: (62,)
person: F01, sentence: 04, n: 02 
(1, 64, 64, 

pad for  18
(1, 18, 64, 64, 3)
27
pad for  18
(1, 18, 64, 64, 3)
(116, 27, 64, 64, 3)
y.shape: (116,)
person: F01, sentence: 06, n: 09 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
(118, 27, 64, 64, 3)
y.shape: (118,)
person: F01, sentence: 06, n: 10 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
(120, 27, 64, 64, 3)
y.shape: (120,)
person: F01, sentence: 07, n: 01 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
(122, 27, 64, 64, 3)
y.shape: (122,)
person: F01, sentence: 07, n

pad for  20
(1, 20, 64, 64, 3)
27
pad for  20
(1, 20, 64, 64, 3)
(174, 27, 64, 64, 3)
y.shape: (174,)
person: F01, sentence: 09, n: 08 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
27
pad for  19
(1, 19, 64, 64, 3)
27
pad for  19
(1, 19, 64, 64, 3)
(176, 27, 64, 64, 3)
y.shape: (176,)
person: F01, sentence: 09, n: 09 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
27
pad for  18
(1, 18, 64, 64, 3)
27
pad for  18
(1, 18, 64, 64, 3)
(178, 27, 64, 64, 3)
y.shape: (178,)
person: F01, sentence: 09, n: 10 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
27
pad for  19
(1, 19, 64, 64, 3)
27
pad for  19
(1, 19, 64, 64, 3)
(180, 27, 64, 64, 3)
y.shape: (180,)
person: F01, sentence: 10, n: 01 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 

27
pad for  13
(1, 13, 64, 64, 3)
(228, 27, 64, 64, 3)
y.shape: (228,)
person: F02, sentence: 02, n: 05 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64, 3)
27
pad for  13
(1, 13, 64, 64, 3)
27
pad for  13
(1, 13, 64, 64, 3)
(230, 27, 64, 64, 3)
y.shape: (230,)
person: F02, sentence: 02, n: 06 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64, 3)
27
pad for  13
(1, 13, 64, 64, 3)
27
pad for  13
(1, 13, 64, 64, 3)
(232, 27, 64, 64, 3)
y.shape: (232,)
person: F02, sentence: 02, n: 07 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13,

...

27
pad for  16
(1, 16, 64, 64, 3)
27
pad for  16
(1, 16, 64, 64, 3)
(2742, 27, 64, 64, 3)
y.shape: (2742,)
person: M07, sentence: 08, n: 02 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
(2744, 27, 64, 64, 3)
y.shape: (2744,)
person: M07, sentence: 08, n: 03 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
27
pad for  14
(1, 14, 64, 64, 3)
27
pad for  14
(1, 14, 64, 64, 3)
(2746, 27, 64, 64, 3)
y.shape: (2746,)
person: M07, sentence: 08, n: 04 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
27
pad for

27
pad for  15
(1, 15, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
(2794, 27, 64, 64, 3)
y.shape: (2794,)
person: M07, sentence: 10, n: 08 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
(2796, 27, 64, 64, 3)
y.shape: (2796,)
person: M07, sentence: 10, n: 09 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
27
pad for  16
(1, 16, 64, 64, 3)
27
pad for  16
(1, 16, 64, 64, 3)
(2798, 27, 64, 64, 3)
y.shape: (2798,)
person: M07, sentence: 10, n: 10 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
27
pad for

27
pad for  15
(1, 15, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
(2848, 27, 64, 64, 3)
y.shape: (2848,)
person: M08, sentence: 03, n: 05 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
27
pad for  15
(1, 15, 64, 64, 3)
(2850, 27, 64, 64, 3)
y.shape: (2850,)
person: M08, sentence: 03, n: 06 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
(2852, 27, 64, 64, 3)
y.shape: (2852,)
person: M08, sentence: 03, n: 07 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
(2854, 27, 64, 64, 3)
y.

27
pad for  19
(1, 19, 64, 64, 3)
27
pad for  19
(1, 19, 64, 64, 3)
(2906, 27, 64, 64, 3)
y.shape: (2906,)
person: M08, sentence: 06, n: 04 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
27
pad for  19
(1, 19, 64, 64, 3)
27
pad for  19
(1, 19, 64, 64, 3)
(2908, 27, 64, 64, 3)
y.shape: (2908,)
person: M08, sentence: 06, n: 05 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
27
pad for  18
(1, 18, 64, 64, 3)
27
pad for  18
(1, 18, 64, 64, 3)
(2910, 27, 64, 64, 3)
y.shape: (2910,)
person: M08, sentence: 06, n: 06 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
27
pad for  18
(1, 18, 64, 64, 3)
27
pad for  18
(1, 18, 64, 64, 3)
(2912, 27, 64, 64, 3)
y.shape: (2912,)
person: M08, sentence: 06, n: 07 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(

27
pad for  13
(1, 13, 64, 64, 3)
27
pad for  13
(1, 13, 64, 64, 3)
(2958, 27, 64, 64, 3)
y.shape: (2958,)
person: M08, sentence: 08, n: 10 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64, 3)
27
pad for  13
(1, 13, 64, 64, 3)
27
pad for  13
(1, 13, 64, 64, 3)
(2960, 27, 64, 64, 3)
y.shape: (2960,)
person: M08, sentence: 09, n: 01 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
27
pad for  17
(1, 17, 64, 64, 3)
(2962, 27, 64, 64, 3)
y.shape: (2962,)
person: M08, sentence: 09, n: 02 
(1, 64, 64, 3)
(2, 64, 64, 3)
(3, 64, 64, 3)
(4, 64, 64, 3)
(5, 64, 64, 3)
(6, 64, 64, 3)
(7, 64, 64, 3)
(8, 64, 64, 3)
(9, 64, 64, 3)
(10, 64, 64, 3)
(11, 64, 64, 3)
(12, 64, 64, 3)
(13, 64, 64, 3)
(14, 64, 64,

In [6]:
np.save('./lip/lip_x_data', x_data) # x_data.npy
np.save('./lip/lip_y_data', y_data) # y_data.npy